## Import Packages

In [ ]:
import os
import gc
import warnings

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Configuration
warnings.simplefilter('ignore')
pd.set_option('max_column', None)
sns.set_style("darkgrid")
colors = sns.color_palette('Set2')

In [ ]:
TRAIN_DIR = "../input/jpx-tokyo-stock-exchange-prediction/train_files"

In [ ]:
%%time

df_prices = pd.read_csv(os.path.join(TRAIN_DIR, 'stock_prices.csv'))
df_prices_sec = pd.read_csv(os.path.join(TRAIN_DIR, 'secondary_stock_prices.csv'))
df_fins = pd.read_csv(os.path.join(TRAIN_DIR, 'financials.csv'))
df_opts = pd.read_csv(os.path.join(TRAIN_DIR, 'options.csv'))
df_trades = pd.read_csv(os.path.join(TRAIN_DIR, 'trades.csv'))
stock_list = pd.read_csv(os.path.join("../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv"))

In [ ]:
def summarize(df, file_name, n_rows_to_show=5):
    """Simply summarize the given DataFrame.
    
    Parameters:
        df: pd.DataFrame, raw DataFrame
        file_name: str, name of the file
        n_rows_to_show: int, number of rows to show 
    """
    print(f"=====Summary of {file_name}=====")
    print(f"Shape: {df.shape}")
    
    nan_ratio = pd.isna(df).sum() / len(df) * 100
    nan_ratio.sort_values(ascending=False, inplace=True)
    nan_ratio = nan_ratio.to_frame(name='NaN Ratio').T
    print("NaN ratio:")
    display(nan_ratio)   
    
    display(df.head(n_rows_to_show))

In [ ]:
summarize(df_prices, "stock_prices.csv")

In [ ]:
row_id = (df_prices['Date'].apply(lambda date: ''.join(date.split('-'))+'_')
          + df_prices['SecuritiesCode'].astype(str))
assert (df_prices['RowId'] == row_id).all()

In [ ]:
n_dates = df_prices['Date'].nunique()
date_min, date_max = df_prices['Date'].min(), df_prices['Date'].max()
n_stocks = df_prices['SecuritiesCode'].nunique()

print(f"Number of unique dates: {n_dates} ({date_min} ~ {date_max})\n"
      f"Number of unique stocks: {n_stocks}")

Over 90% of stocks have records for complete training period from `2017-01-04` to `2021-12-03`. However, we still need to pay attention to **missing values** in certain columns (*e.g.*, `ExpectedDividend`, price columns).

In [ ]:
n_dates_per_stock = df_prices.groupby(['SecuritiesCode'])['Date'].count()
stock_cnt_max = (n_dates_per_stock == n_dates_per_stock.max()).sum()
stock_ratio_max = stock_cnt_max / len(n_dates_per_stock) * 100

fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=n_dates_per_stock.values, bins=100, palette=colors, ax=ax)
ax.axhline(y=stock_cnt_max, color='red', linestyle='dotted', linewidth=2,
           label=f'{stock_ratio_max}% of #Stocks')
ax.set_title(f"Number of Dates per Stock\nMin {n_dates_per_stock.min()} | "
             f"Max {n_dates_per_stock.max()}")
ax.set_xlabel("#Dates")
ax.set_ylabel("Stock Count")
ax.legend()
plt.show()

In [ ]:
n_stocks_per_date = df_prices.groupby(['Date'])['SecuritiesCode'].count()
date_cnt_max = (n_stocks_per_date == n_stocks_per_date.max()).sum()
date_ratio_max = round(date_cnt_max / len(n_stocks_per_date) * 100, 2)

fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=n_stocks_per_date.values, bins=100, palette=colors, ax=ax)
ax.axvline(x=n_stocks_per_date.mean(), color='orange', linestyle='dotted',
           linewidth=2, label='Mean')
ax.axvline(x=n_stocks_per_date.median(), color='green', linestyle='--',
           linewidth=2, label='Median')
ax.axhline(y=date_cnt_max, color='red', linestyle='dotted', linewidth=2,
           label=f'{date_ratio_max}% of #Dates')
ax.set_title(f"Number of Stocks per Date\nMin {n_stocks_per_date.min()} | "
             f"Max {n_stocks_per_date.max()}")
ax.set_xlabel("#Stocks")
ax.set_ylabel("Date Count")
ax.legend()
plt.show()

In [ ]:
df_no_prices = df_prices[df_prices['Close'].isna()]
print(f"Number of samples without prices: {len(df_no_prices)}")

In [ ]:
no_prices_per_date = df_no_prices['Date'].value_counts().sort_index()
all_dates = sorted(df_prices['Date'].unique())
no_prices_per_date = no_prices_per_date.reindex(all_dates, fill_value=0)

fig = go.Figure()
fig.add_trace(go.Scatter(x=no_prices_per_date.index, 
                         y=no_prices_per_date, 
                         mode='lines'))
rect_focus_params = {
    'xref': 'x', 'yref': 'y', 'x0': '2020-09-25', 'x1': '2020-10-08',
    'y0': 1900, 'y1': 2000, 'line': {'color': 'red', 'width': 1.5}
}
fig.add_shape(type='rect', **rect_focus_params)
fig.update_layout(
    title=f"Number of Stocks without Prices per Date",
    xaxis_title="Date",
    yaxis_title="Stock Count",
)
fig.show()

In [ ]:
assert (len(df_prices[df_prices['Date'] == '2020-10-01']) 
        == no_prices_per_date.max())

In [ ]:
no_prices_per_stock = df_no_prices['SecuritiesCode'].value_counts()
stock_cnt_max = (no_prices_per_stock == no_prices_per_stock.min()).sum()
stock_ratio_max = round(stock_cnt_max / len(no_prices_per_stock) * 100, 2)

fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=no_prices_per_stock.values, bins=100, palette=colors, ax=ax)
ax.axhline(y=stock_cnt_max, color='red', linestyle='dotted', linewidth=2,
           label=f'{stock_ratio_max}% of #Stocks')
ax.set_title(f"Number of Dates w/o Prices per Stock\n"
             f"Min {no_prices_per_stock.min()} | "
             f"Max {no_prices_per_stock.max()}")
ax.set_xlabel("#Dates without Prices")
ax.set_ylabel("Stock Count")
ax.legend()
plt.show()

In [ ]:
def plot_candle_with_target(stock_code, prime=True):
    """Plot OHLCV plot with target series.
    
    Parameters:
        stock_code: int, code of the stock
        prime: bool, whether the stock to plot is prime or not
    """
    df_ = df_prices.copy() if prime else df_prices_sec.copy()
    df_ = df_[df_['SecuritiesCode'] == stock_code]
    dates = df_['Date'].values
    ohlc = {
        'open': df_['Open'].values, 
        'high': df_['High'].values, 
        'low': df_['Low'].values, 
        'close': df_['Close'].values
    }
    vol = df_['Volume'].values
    target = df_['Target'].values
    
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, x_title='Date')
    fig.add_trace(go.Candlestick(x=dates, name='OHLC', **ohlc),
                  row=1, col=1)
    fig.add_trace(go.Bar(x=dates, y=vol, name='Volume'),
                  row=2, col=1)
    fig.add_trace(go.Scatter(x=dates, y=target, name='Target'),
                  row=3, col=1)
    fig.update_layout(
        title=f"OHLCV Chart with Target Series (Stock {stock_code})",
    )
    fig.update(layout_xaxis_rangeslider_visible=False)
    fig.show()

In [ ]:
for stock_code in df_prices['SecuritiesCode'].unique()[:3]:
    plot_candle_with_target(stock_code)

In [ ]:
target_mean_per_stock = df_prices.groupby(['SecuritiesCode'])['Target'].mean()
target_mean_mean = target_mean_per_stock.mean()

fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=target_mean_per_stock.values, bins=100, palette=colors, kde=True,
             ax=ax)
ax.axvline(x=target_mean_mean, color='orange', linestyle='dotted', linewidth=2, 
           label='Mean')
ax.set_title("Target Mean Distibution\n"
             f"Min {round(target_mean_per_stock.min(), 4)} | "
             f"Max {round(target_mean_per_stock.max(), 4)} | "
             f"Skewness {round(target_mean_per_stock.skew(), 2)} | "
             f"Kurtosis {round(target_mean_per_stock.kurtosis(), 2)}")
ax.set_xlabel("Target Mean")
ax.set_ylabel("Stock Count")
ax.legend()
plt.show()

In [ ]:
target_std_per_stock = df_prices.groupby(['SecuritiesCode'])['Target'].std()
target_std_mean = target_std_per_stock.mean()

fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=target_std_per_stock.values, bins=100, palette=colors, kde=True,
             ax=ax)
ax.axvline(x=target_std_mean, color='orange', linestyle='dotted', linewidth=2, 
           label='Mean')
ax.set_title("Target Std Distibution\n"
             f"Min {round(target_std_per_stock.min(), 4)} | "
             f"Max {round(target_std_per_stock.max(), 4)} | "
             f"Skewness {round(target_std_per_stock.skew(), 2)} | "
             f"Kurtosis {round(target_std_per_stock.kurtosis(), 2)}")
ax.set_xlabel("Target Std")
ax.set_ylabel("Stock Count")
ax.legend()
plt.show()

In [ ]:
ax = sns.jointplot(x=n_dates_per_stock, y=target_mean_per_stock, 
                   kind="reg", height=6, marginal_ticks=True, 
                   joint_kws={'line_kws':{'color':'orange'}})
ax.ax_joint.set_xlabel('#Dates per Stock')
ax.ax_joint.set_ylabel('Target Mean')
plt.show()

In [ ]:
ax = sns.jointplot(x=n_dates_per_stock, y=target_std_per_stock, 
                   kind="reg", height=6, marginal_ticks=True, 
                   joint_kws={'line_kws':{'color':'orange'}})
ax.ax_joint.set_xlabel('#Dates per Stock')
ax.ax_joint.set_ylabel('Target Std')
plt.show()

In [ ]:
n_dates_per_stock_ = n_dates_per_stock[n_dates_per_stock != 1202]
target_mean_per_stock_ = target_mean_per_stock[target_mean_per_stock.index.isin(n_dates_per_stock_.index)]

ax = sns.jointplot(x=n_dates_per_stock_, y=target_mean_per_stock_, 
                   kind="reg", height=6, marginal_ticks=True, 
                   joint_kws={'line_kws':{'color':'orange'}})
ax.ax_joint.set_xlabel('#Dates per Stock (#Dates != 1202)')
ax.ax_joint.set_ylabel('Target Mean')
plt.show()

In [ ]:
target_std_per_stock_ = target_std_per_stock[target_std_per_stock.index.isin(n_dates_per_stock_.index)]

ax = sns.jointplot(x=n_dates_per_stock_, y=target_std_per_stock_, 
                   kind="reg", height=6, marginal_ticks=True, 
                   joint_kws={'line_kws':{'color':'orange'}})
ax.ax_joint.set_xlabel('#Dates per Stock (#Dates != 1202)')
ax.ax_joint.set_ylabel('Target Std')
plt.show()

In [ ]:
tm_max = target_mean_per_stock.max()
stock_tm_max = target_mean_per_stock[target_mean_per_stock == tm_max].index.values[0]

print(f"Stock with max target mean is {stock_tm_max}.")
plot_candle_with_target(stock_tm_max)

In [ ]:
target_mean_per_date = df_prices.groupby(['Date'])['Target'].mean()
target_mean_mean = target_mean_per_date.mean()

fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=target_mean_per_date.values, bins=100, palette=colors, kde=True,
             ax=ax)
ax.axvline(x=target_mean_mean, color='orange', linestyle='dotted', linewidth=2, 
           label='Mean')
ax.set_title("Target Mean Distibution\n"
             f"Min {round(target_mean_per_date.min(), 4)} | "
             f"Max {round(target_mean_per_date.max(), 4)} | "
             f"Skewness {round(target_mean_per_date.skew(), 2)} | "
             f"Kurtosis {round(target_mean_per_date.kurtosis(), 2)}")
ax.set_xlabel("Target Mean")
ax.set_ylabel("Date Count")
ax.legend()
plt.show()

In [ ]:
target_std_per_date = df_prices.groupby(['Date'])['Target'].std()
target_std_mean = target_std_per_date.mean()

fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=target_std_per_date.values, bins=100, palette=colors, kde=True,
             ax=ax)
ax.axvline(x=target_std_mean, color='orange', linestyle='dotted', linewidth=2, 
           label='Mean')
ax.set_title("Target Std Distibution\n"
             f"Min {round(target_std_per_date.min(), 4)} | "
             f"Max {round(target_std_per_date.max(), 4)} | "
             f"Skewness {round(target_std_per_date.skew(), 2)} | "
             f"Kurtosis {round(target_std_per_date.kurtosis(), 2)}")
ax.set_xlabel("Target Std")
ax.set_ylabel("Date Count")
ax.legend()
plt.show()

In [ ]:
zero_target_std = target_std_per_date[target_std_per_date == 0].index.values
print(f"The trading dates with zero target std is {zero_target_std}.")

dates_after = df_prices[(df_prices['Date'] >= zero_target_std[0]) & 
                        (df_prices['Date'] < '2020-10-02')]['Date'].unique()
print(f"T+1 trading date of that date is {dates_after[1]}")
print(f"T+2 trading date of that date is {dates_after[2]}")

In [ ]:
ax = sns.jointplot(x=n_stocks_per_date, y=target_mean_per_date, 
                   kind="reg", height=6, marginal_ticks=True, 
                   joint_kws={'line_kws':{'color':'orange'}})
ax.ax_joint.set_xlabel('#Stocks per Date')
ax.ax_joint.set_ylabel('Target Mean')
plt.show()

In [ ]:
ax = sns.jointplot(x=n_stocks_per_date, y=target_std_per_date, 
                   kind="reg", height=6, marginal_ticks=True, 
                   joint_kws={'line_kws':{'color':'orange'}})
ax.ax_joint.set_xlabel('#Stocks per Date')
ax.ax_joint.set_ylabel('Target Std')
plt.show()

In [ ]:
tm_max = target_mean_per_date.max()
date_tm_max = target_mean_per_date[target_mean_per_date == tm_max].index.values[0]
print(f"Date with max target mean is {date_tm_max}.")

In [ ]:
ts_max = target_std_per_date.max()
date_ts_max = target_std_per_date[target_std_per_date == ts_max].index.values[0]
print(f"Date with max target std is {date_ts_max}.")

In [ ]:
summarize(stock_list, "stock_list.csv", n_rows_to_show=2)

In [ ]:
assert stock_list['Universe0'].sum() == 2000

In [ ]:
sec_prod_val_cnt = stock_list['Section/Products'].value_counts()
pct = sec_prod_val_cnt / sec_prod_val_cnt.sum() * 100
labels = [f"{sec} {ratio:.2f}%" for sec, ratio in zip(sec_prod_val_cnt.index, pct)]

fig, ax = plt.subplots(figsize=(10, 5))
patches, texts = ax.pie(sec_prod_val_cnt.values, 
                        colors=colors, 
                        shadow=True, 
                        startangle=90)
patches, labels, dummy = zip(*sorted(zip(patches, labels, sec_prod_val_cnt.values),
                                     key=lambda x: x[2],
                                     reverse=True))
ax.legend(patches, labels, bbox_to_anchor=(-0.1, 1.), fontsize=8)
ax.set_title("Ratio of Section/Products")
plt.show()

In [ ]:
target_stock_list = stock_list[stock_list['Universe0']]
sec_prod_val_cnt = target_stock_list['Section/Products'].value_counts()
pct = sec_prod_val_cnt / sec_prod_val_cnt.sum() * 100
labels = [f"{sec} {ratio:.2f}%" for sec, ratio in zip(sec_prod_val_cnt.index, pct)]

fig, ax = plt.subplots(figsize=(10, 5))
patches, texts = ax.pie(sec_prod_val_cnt.values, 
                        colors=sns.color_palette("pastel"), 
                        shadow=True, 
                        startangle=90)
patches, labels, dummy = zip(*sorted(zip(patches, labels, sec_prod_val_cnt.values),
                                     key=lambda x: x[2],
                                     reverse=True))
ax.legend(patches, labels, bbox_to_anchor=(-0.1, 1.), fontsize=8)
ax.set_title("Ratio of Section/Products (2000 Target Stocks)")
plt.show()

In [ ]:
sec_33_val_cnt = target_stock_list['33SectorName'].value_counts()
pct = sec_33_val_cnt / sec_33_val_cnt.sum() * 100
labels = [f"{sec} {ratio:.2f}%" for sec, ratio in zip(sec_33_val_cnt.index, pct)]

fig, ax = plt.subplots(figsize=(10, 5))
patches, texts = ax.pie(sec_33_val_cnt.values, 
                        colors=sns.color_palette("pastel"), 
                        shadow=True, 
                        startangle=90)
patches, labels, dummy = zip(*sorted(zip(patches, labels, sec_33_val_cnt.values),
                                     key=lambda x: x[2],
                                     reverse=True))
ax.legend(patches, labels, bbox_to_anchor=(-0.1, 1.), fontsize=8)
ax.set_title("Ratio of 33 Sector Name (2000 Target Stocks)")
plt.show()

In [ ]:
sec_17_val_cnt = target_stock_list['17SectorName'].value_counts()
pct = sec_17_val_cnt / sec_17_val_cnt.sum() * 100
labels = [f"{sec} {ratio:.2f}%" for sec, ratio in zip(sec_17_val_cnt.index, pct)]

fig, ax = plt.subplots(figsize=(10, 5))
patches, texts = ax.pie(sec_17_val_cnt.values, 
                        colors=sns.color_palette("pastel"), 
                        shadow=True, 
                        startangle=90)
patches, labels, dummy = zip(*sorted(zip(patches, labels, sec_33_val_cnt.values),
                                     key=lambda x: x[2],
                                     reverse=True))
ax.legend(patches, labels, bbox_to_anchor=(-0.1, 1.), fontsize=8)
ax.set_title("Ratio of TOPIX-17 Series Name (2000 Target Stocks)")
plt.show()

In [ ]:
sec_info = target_stock_list[['SecuritiesCode', '33SectorName', '17SectorName']]
df_prices = pd.merge(df_prices, sec_info, on='SecuritiesCode')
df_prices.head()

In [ ]:
target_mean_33sec = df_prices.groupby(['33SectorName'])['Target'].mean()
target_mean_33sec.sort_values(inplace=True, ascending=False)

fig, ax = plt.subplots(figsize=(10, 5))
sns.barplot(x=target_mean_33sec.index, y=target_mean_33sec.values, 
            palette=colors, ax=ax)
ax.set_title("Target Mean of Each Sector over Training Period")
ax.set_xlabel("33 Sector Name")
ax.tick_params(axis='x', rotation=90)
ax.set_ylabel("Target Mean")
plt.show()

In [ ]:
target_std_33sec = df_prices.groupby(['33SectorName'])['Target'].std()
target_std_33sec.sort_values(inplace=True, ascending=False)

fig, ax = plt.subplots(figsize=(10, 5))
sns.barplot(x=target_std_33sec.index, y=target_std_33sec.values, 
            palette=colors, ax=ax)
ax.set_title("Target Std of Each Sector over Training Period")
ax.set_xlabel("33 Sector Name")
ax.tick_params(axis='x', rotation=90)
ax.set_ylabel("Target Std")
plt.show()

In [ ]:
target_sector = 'Marine Transportation'
df_prices_marine = df_prices[df_prices['33SectorName'] == target_sector]
target_mean_marine = df_prices_marine.groupby(['SecuritiesCode'])['Target'].mean()
target_mean_marine.sort_values(inplace=True, ascending=False)

fig, ax = plt.subplots(figsize=(10, 5))
sns.barplot(x=target_mean_marine.index, y=target_mean_marine.values, 
            order=target_mean_marine.index, palette=colors, ax=ax)
ax.set_title("Target Mean of Stocks in Sector Marine Transportation over Training Period")
ax.set_xlabel("Security Code")
ax.set_ylabel("Target Mean")
plt.show()

In [ ]:
unique_dates = sorted(df_prices['Date'].unique())

fig = go.Figure()
for stock_code in [9101, 9107, 9104]:
    df_ = df_prices_marine[df_prices_marine['SecuritiesCode'] == stock_code]
    fig.add_trace(go.Scatter(x=unique_dates, y=df_['Target'], name=stock_code))
fig.update_layout(
    title=f"Target Series of 3 Best Stocks in Marine Transportation",
)
fig.show()

In [ ]:
target_sector = 'Banks'
df_prices_banks = df_prices[df_prices['33SectorName'] == target_sector]
target_mean_banks = df_prices_banks.groupby(['SecuritiesCode'])['Target'].mean()
target_mean_banks.sort_values(inplace=True, ascending=False)

fig, ax = plt.subplots(figsize=(10, 5))
sns.barplot(x=target_mean_banks.index, y=target_mean_banks.values, 
            order=target_mean_banks.index, palette=colors, ax=ax)
ax.set_title("Target Mean of Stocks in Sector Banks over Training Period")
ax.set_xlabel("Security Code")
ax.tick_params(axis='x', rotation=90)
ax.set_ylabel("Target Mean")
plt.show()

In [ ]:
fig = go.Figure()
for stock_code in [8360, 8714, 8358]:
    df_ = df_prices_banks[df_prices_banks['SecuritiesCode'] == stock_code]
    fig.add_trace(go.Scatter(x=unique_dates, y=df_['Target'], name=stock_code))
fig.update_layout(
    title=f"Target Series of 3 Worst Stocks in Banks",
)
fig.show()

In [ ]:
fig = go.Figure()
for stock_code in [9101, 9104, 8714, 8358]:
    df_ = df_prices[df_prices['SecuritiesCode'] == stock_code]
    fig.add_trace(go.Scatter(x=unique_dates, y=df_['Target'], name=stock_code))
fig.update_layout(
    title=f"Target Series of 2 Best Stocks in Marine Transportation and 2 Worst in Banks",
)
fig.show()

In [ ]:
summarize(df_prices_sec, "secondary_stock_prices.csv")

In [ ]:
row_id = (df_prices_sec['Date'].apply(lambda date: ''.join(date.split('-'))+'_')
          + df_prices_sec['SecuritiesCode'].astype(str))
assert (df_prices_sec['RowId'] == row_id).all()

In [ ]:
n_dates = df_prices_sec['Date'].nunique()
date_min, date_max = df_prices_sec['Date'].min(), df_prices['Date'].max()
n_stocks = df_prices_sec['SecuritiesCode'].nunique()

print(f"Number of unique dates: {n_dates} ({date_min} ~ {date_max})\n"
      f"Number of unique secondary stocks: {n_stocks}")

In [ ]:
target_mean_per_stock = df_prices_sec.groupby(['SecuritiesCode'])['Target'].mean()
target_mean_mean = target_mean_per_stock.mean()

fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=target_mean_per_stock.values, bins=100, palette=colors, kde=True,
             ax=ax)
ax.axvline(x=target_mean_mean, color='orange', linestyle='dotted', linewidth=2, 
           label='Mean')
ax.set_title("Target Mean Distibution of Secondary Stocks\n"
             f"Min {round(target_mean_per_stock.min(), 4)} | "
             f"Max {round(target_mean_per_stock.max(), 4)} | "
             f"Skewness {round(target_mean_per_stock.skew(), 2)} | "
             f"Kurtosis {round(target_mean_per_stock.kurtosis(), 2)}")
ax.set_xlabel("Target Mean")
ax.set_ylabel("Stock Count")
ax.legend()
plt.show()

In [ ]:
target_std_per_stock = df_prices_sec.groupby(['SecuritiesCode'])['Target'].std()
target_std_mean = target_std_per_stock.mean()

fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=target_std_per_stock.values, bins=100, palette=colors, kde=True,
             ax=ax)
ax.axvline(x=target_std_mean, color='orange', linestyle='dotted', linewidth=2, 
           label='Mean')
ax.set_title("Target Std Distibution of Secondary Stocks\n"
             f"Min {round(target_std_per_stock.min(), 4)} | "
             f"Max {round(target_std_per_stock.max(), 4)} | "
             f"Skewness {round(target_std_per_stock.skew(), 2)} | "
             f"Kurtosis {round(target_std_per_stock.kurtosis(), 2)}")
ax.set_xlabel("Target Std")
ax.set_ylabel("Stock Count")
ax.legend()
plt.show()

In [ ]:
target_mean_per_stock.sort_values(inplace=True, ascending=False)
target_std_per_stock.sort_values(inplace=True, ascending=False)
stock_with_max_tm = target_mean_per_stock.index[0]
stock_with_min_tm = target_mean_per_stock.index[-1]
stock_with_max_ts = target_std_per_stock.index[0]
stock_with_min_ts = target_std_per_stock.index[-1]

print(f"The secondary stock with the max target mean is {stock_with_max_tm},"
      f" and min is {stock_with_min_tm}.")
print(f"The secondary stock with the max target std is {stock_with_max_ts},"
      f" and min is {stock_with_min_ts}.")

In [ ]:
for stock_code in [4412, 9252, 1689, 8728]:
    plot_candle_with_target(stock_code, prime=False)

In [ ]:
summarize(df_trades, 'trades.csv')

In [ ]:
nan_cnt = df_trades.isna().sum().max()
df_trades.dropna(inplace=True)
assert len(df_trades) == 1712 - nan_cnt

In [ ]:
section_cnt = df_trades['Section'].value_counts()

fig, ax = plt.subplots(figsize=(10, 5))
sns.barplot(x=section_cnt.index, y=section_cnt.values,  palette=colors, ax=ax)
ax.set_title("Section Count")
ax.set_xlabel("Section Name")
ax.set_ylabel("Count")
plt.show()

In [ ]:
def show_ts_in_diff_sec(feat_name):
    """Plot time series of the given feature in different sections.
    
    Parameters:
        feat_name: str, feature name 
    """
    fig = px.line(df_trades, x='EndDate', y=feat_name, color='Section', 
                  symbol='Section')
    fig.update_layout(
        title=f"{feat_name} Series",
    )
    fig.show()

In [ ]:
for feat in ['TotalSales', 'TotalPurchases']:
    show_ts_in_diff_sec(feat)

In [ ]:
summarize(df_opts, 'options.csv')

In [ ]:
date_code = (df_opts['Date'].apply(lambda date: ''.join(date.split('-'))+'_')
             + df_opts['OptionsCode'].astype(str))
assert (df_opts['DateCode'] == date_code).all()

In [ ]:
n_dates = df_opts['Date'].nunique()
date_min, date_max = df_opts['Date'].min(), df_opts['Date'].max()
n_opts = df_opts['OptionsCode'].nunique()

print(f"Number of unique dates: {n_dates} ({date_min} ~ {date_max})\n"
      f"Number of unique options: {n_opts}")

In [ ]:
n_dates_per_opt = df_opts.groupby(['OptionsCode'])['Date'].count()
n_dates_per_opt_max = n_dates_per_opt.max()

fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=n_dates_per_opt.values, bins=100, palette=colors, ax=ax)
ax.axvline(x=n_dates_per_opt_max, color='orange', linestyle='dotted', linewidth=2,
           label=f'Max #Dates: {n_dates_per_opt_max}')
ax.set_title(f"Number of Dates per Option\nMin {n_dates_per_opt.min()} | "
             f"Max {n_dates_per_opt_max}")
ax.set_xlabel("#Dates")
ax.set_ylabel("Option Count")
ax.legend()
plt.show()

In [ ]:
n_opts_per_date = df_opts.groupby(['Date'])['OptionsCode'].count()

fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=n_opts_per_date.values, bins=100, palette=colors, ax=ax)
ax.set_title(f"Number of Options per Date\nMin {n_opts_per_date.min()} | "
             f"Max {n_opts_per_date.max()}")
ax.set_xlabel("#Options")
ax.set_ylabel("Date Count")
plt.show()

In [ ]:
zero_ratio = (df_opts == 0).sum() / len(df_opts) * 100
zero_ratio.sort_values(ascending=False, inplace=True)

fig, ax = plt.subplots(figsize=(10, 5))
sns.barplot(x=zero_ratio.index, y=zero_ratio.values, palette=colors, ax=ax)
ax.set_title("Zero Ratio of Options Data")
ax.set_xlabel("Column Name")
ax.tick_params(axis='x', rotation=90)
ax.set_ylabel("Zero Ratio")
plt.show()

In [ ]:
opts = df_opts['OptionsCode'].unique()
print(f"Number of unique options: {len(opts)}")

In [ ]:
assert np.sum((opts // 1e8) == 0) == 0

In [ ]:
deriv_class, deriv_val_cnt = np.unique(opts % 1e8 // 1e7, return_counts=True)
pct = deriv_val_cnt / np.sum(deriv_val_cnt) * 100
labels = [f"{int(c)} - {ratio:.2f}%" for c, ratio in zip(deriv_class, pct)]

fig, ax = plt.subplots(figsize=(10, 5))
patches, texts = ax.pie(deriv_val_cnt, 
                        colors=colors, 
                        shadow=True, 
                        startangle=90)
patches, labels, dummy = zip(*sorted(zip(patches, labels, deriv_val_cnt),
                                     key=lambda x: x[2],
                                     reverse=True))
ax.legend(patches, labels, bbox_to_anchor=(-0.1, 1.), fontsize=8)
ax.set_title("Derivative Class")
plt.show()

In [ ]:
# Convert contract year to 10-year cycle code
year2code = lambda y: ((y%10) + 5) % 10

In [ ]:
df_opts_exp = df_opts[['OptionsCode', 'ContractMonth']]
df_opts_exp['ContractMonth'] = pd.to_datetime(df_opts_exp['ContractMonth'],
                                              format="%Y%m")
df_opts_exp['ExpYear'] = df_opts_exp['ContractMonth'].dt.year
df_opts_exp['ExpMonth'] = df_opts_exp['ContractMonth'].dt.month
df_opts_exp['ExpYearCode'] = df_opts_exp['ExpYear'].apply(year2code)
df_opts_exp.drop(['ContractMonth'], inplace=True, axis=1)

df_opts_exp.head()

In [ ]:
assert ((df_opts_exp['OptionsCode'] % 1e7 // 1e6) 
        != df_opts_exp['ExpYearCode']).any() == 0

In [ ]:
exp_prices = (df_opts['OptionsCode'] % 1e4 // 1e2).value_counts()

fig, ax = plt.subplots(figsize=(14, 7))
sns.barplot(x=exp_prices.index.astype(int), y=exp_prices.values, 
            order=exp_prices.index.astype(int), palette=colors, ax=ax)
ax.set_title("Strike Price Distribution")
ax.set_xlabel("Strike Price")
ax.tick_params(axis='x', rotation=90)
ax.set_ylabel("Count")
plt.show()

In [ ]:
df_opts['UnderlyingIndex'] = (df_opts['OptionsCode'] % 1e2).astype(np.int8)
unique_underlying = df_opts['UnderlyingIndex'].unique()

print(f"Number of underlying index: {len(unique_underlying)}\n"
      f"Their codes are {unique_underlying}")

[![1111.jpg](https://i.postimg.cc/T3K8qkCV/1111.jpg)](https://postimg.cc/qNHZpGfq)